# Snowflake Multi-Table Dataset

In this notebook, we will learn how to train a classifier with a more complex multi-table data where a secondary table is itself a parent tables of another table (ie. snowflake schema). It is highly recommended to see the _Sklearn Basics 1_ and _Sklearn Basics 2_ lessons if you are not familiar with pyKhiops' sklearn estimators.

We start by importing pyKhiops sklearn classifier `KhiopsClassifier` and saving the location of the Khiops `Samples` directory into a variable:

In [1]:
from os import path
import pandas as pd

from khiops import core as kh
from khiops.sklearn import KhiopsClassifier

/opt/homebrew/Caskroom/miniconda/base/envs/benchmark/lib/python3.10/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


### Training a Multi-Table Classifier

We'll train a multi-table classifier on a extension of dataset `AccidentsSummary` that we used in the previous notebook Sklearn Basics 2. This dataset `Accidents` contains the additional table `Users` and is organized in the following relational snowflake schema.

```
Accidents
|
| -- 1:n -- Vehicles
              |
              |-- 1:n -- Users
```

Note that the target variable is `Gravity`.

To train the KhiopsClassifier for this setup, we must specify a multi-table dataset. Let's first check the content of the tables:

- The main table `Accidents`
- The secondary table `Vehicles`
- The tertiary table `Users`


In [2]:
accidents_dataset_dir = path.join(kh.get_samples_dir(), "Accidents")

accidents_file = path.join(
    path.join(kh.get_samples_dir(), "AccidentsSummary"), "Accidents.txt"
)
accidents_df = pd.read_csv(accidents_file, sep="\t", encoding="latin1")
print(f"Accident dataframe (first 10 rows):")
display(accidents_df.head(10))
print()

# We drop the "Gravity" column as it was used to create the target
users_file = path.join(accidents_dataset_dir, "Users.txt")
users_df = pd.read_csv(users_file, sep="\t", encoding="latin1").drop("Gravity", axis=1)
print(f"User dataframe (first 10 rows):")
display(users_df.head(10))
print()

vehicles_file = path.join(accidents_dataset_dir, "Vehicles.txt")
vehicles_df = pd.read_csv(vehicles_file, sep="\t", encoding="latin1")
print(f"Vehicle dataframe (first 10 rows):")
display(vehicles_df.head(10))

Accident dataframe (first 10 rows):


,AccidentId,Gravity,Date,Hour,Light,Department,Commune,InAgglomeration,IntersectionType,Weather,CollisionType,PostalAddress
0,201800000001,NonLethal,2018-01-24,15:05:00,Daylight,590,5,No,Y-type,Normal,2Vehicles-BehindVehicles-Frontal,route des Ansereuilles
1,201800000002,NonLethal,2018-02-12,10:15:00,Daylight,590,11,Yes,Square,VeryGood,NoCollision,Place du général de Gaul
2,201800000003,NonLethal,2018-03-04,11:35:00,Daylight,590,477,Yes,T-type,Normal,NoCollision,Rue nationale
3,201800000004,NonLethal,2018-05-05,17:35:00,Daylight,590,52,Yes,NoIntersection,VeryGood,2Vehicles-Side,30 rue Jules Guesde
4,201800000005,NonLethal,2018-06-26,16:05:00,Daylight,590,477,Yes,NoIntersection,Normal,2Vehicles-Side,72 rue Victor Hugo
5,201800000006,NonLethal,2018-09-23,06:30:00,TwilightOrDawn,590,52,Yes,NoIntersection,LightRain,Other,D39
6,201800000007,NonLethal,2018-09-26,00:40:00,NightStreelightsOn,590,133,Yes,NoIntersection,Normal,Other,4 route de camphin
7,201800000008,Lethal,2018-11-30,17:15:00,NightStreelightsOn,590,11,Yes,NoIntersection,Normal,Other,rue saint exupéry
8,201800000009,NonLethal,2018-02-18,15:57:00,Daylight,590,550,No,NoIntersection,Normal,Other,rue de l'égalité
9,201800000010,NonLethal,2018-03-19,15:30:00,Daylight,590,51,Yes,X-type,Normal,2Vehicles-BehindVehicles-Frontal,face au 59 rue de Lille



User dataframe (first 10 rows):


,AccidentId,VehicleId,Seat,Category,Gender,TripReason,SafetyDevice,SafetyDeviceUsed,PedestrianLocation,PedestrianAction,PedestrianCompany,BirthYear
0,201800000001,A01,1.0,Driver,Male,Leisure,SeatBelt,Yes,NaN,NaN,Unknown,1960.0
1,201800000001,B01,1.0,Driver,Male,NaN,SeatBelt,Yes,NaN,NaN,Unknown,1928.0
2,201800000002,A01,1.0,Driver,Male,NaN,SeatBelt,Yes,NaN,NaN,Unknown,1947.0
3,201800000002,A01,NaN,Pedestrian,Male,NaN,Helmet,NaN,OnLane<=OnSidewalk0mCrossing,Crossing,Alone,1959.0
4,201800000003,A01,1.0,Driver,Male,Leisure,Helmet,Yes,NaN,NaN,Unknown,1987.0
5,201800000003,C01,1.0,Driver,Male,NaN,ChildrenDevice,NaN,NaN,NaN,Unknown,1977.0
6,201800000004,A01,1.0,Driver,Male,Leisure,SeatBelt,Yes,NaN,NaN,Unknown,1982.0
7,201800000004,B01,1.0,Driver,Male,Leisure,Helmet,NaN,NaN,NaN,Unknown,2013.0
8,201800000005,A01,1.0,Driver,Male,Leisure,Helmet,Yes,NaN,NaN,Unknown,2001.0
9,201800000005,B01,1.0,Driver,Male,Leisure,SeatBelt,Yes,NaN,NaN,Unknown,1946.0



Vehicle dataframe (first 10 rows):


,AccidentId,VehicleId,Direction,Category,PassengerNumber,FixedObstacle,MobileObstacle,ImpactPoint,Maneuver
0,201800000001,A01,Unknown,Car<=3.5T,0,NaN,Vehicle,RightFront,TurnToLeft
1,201800000001,B01,Unknown,Car<=3.5T,0,NaN,Vehicle,LeftFront,NoDirectionChange
2,201800000002,A01,Unknown,Car<=3.5T,0,NaN,Pedestrian,NaN,NoDirectionChange
3,201800000003,A01,Unknown,Motorbike>125cm3,0,StationaryVehicle,Vehicle,Front,NoDirectionChange
4,201800000003,B01,Unknown,Car<=3.5T,0,NaN,Vehicle,LeftSide,TurnToLeft
5,201800000003,C01,Unknown,Car<=3.5T,0,NaN,NaN,RightSide,Parked
6,201800000004,A01,Unknown,Car<=3.5T,0,NaN,Other,RightFront,Avoidance
7,201800000004,B01,Unknown,Bicycle,0,NaN,Vehicle,LeftSide,NaN
8,201800000005,A01,Unknown,Moped,0,NaN,Vehicle,RightFront,PassLeft
9,201800000005,B01,Unknown,Car<=3.5T,0,NaN,Vehicle,LeftFront,Park


#### Create the main feature matrix and the target vector for `Accidents`

In [3]:
accidents_main_df = accidents_df.drop("Gravity", axis=1)
y_accidents_train = accidents_df["Gravity"]

#### Create the multi-table dataset specification

Note the main table has one key `AccidentId`. Secondary table `Places` has the same key.
Tables `Vehicles` (the other secondary table) and `Users` (the tertiary table) have two keys `AccidentId` and `VehicleId`.

To describe relations between tables, the field `relations` must be added to the dictionary of table specifications. This field
is a list of pairs of tables of the form 
```
(<parent table name>, <child table name>)
``` 

Note that `Places` is here considered as a table as `pykhiops.sklearn` estimators do not support 1:1 relationships at the moment. 

In [4]:
X_accidents_train = {
    "main_table": "Accidents",
    "tables": {
        "Accidents": (accidents_main_df, "AccidentId"),
        "Vehicles": (vehicles_df, ["AccidentId", "VehicleId"]),
        "Users": (users_df, ["AccidentId", "VehicleId"]),
    },
    "relations": [
        ("Accidents", "Vehicles"),
        ("Vehicles", "Users"),
    ],
}

#### Train a classifier with this dataset

- You may choose the number of features `n_features` to be created by the Khiops AutoML engine
- Set the number of trees to zero (`n_trees=0`)

In [5]:
pkc_accidents = KhiopsClassifier(n_trees=0, n_features=1000)
pkc_accidents.fit(X_accidents_train, y_accidents_train)

KhiopsClassifier(n_features=1000, n_trees=0)

#### Print the accuracy and auc of the model


In [6]:
accidents_train_performance = (
    pkc_accidents.model_report_.train_evaluation_report.get_snb_performance()
)
print(f"Accidents train accuracy: {accidents_train_performance.accuracy}")
print(f"Accidents train auc     : {accidents_train_performance.auc}")

Accidents train accuracy: 0.94488
Accidents train auc     : 0.842232


#### Deploy the classifier to obtain predictions on the training data

Note that usually one deploys the model on new test data. We deploy on the train dataset to keep the tutorial simple*.


In [7]:
pkc_accidents.predict(X_accidents_train)

array(['NonLethal', 'NonLethal', 'NonLethal', ..., 'NonLethal',
       'NonLethal', 'NonLethal'], dtype='<U9')